In [1]:
import itertools
from gurobipy import Model, GRB

from examples import Examples

examples = Examples()

def create_hypergraph(attributes, relations):
    nodes = []
    nodes.extend(attributes)
    for rel in relations:
        nodes.extend(rel)
    nodes = list(set(nodes))
    hyperedges = []
    for rel in relations:
        hyperedges.append(frozenset(rel))
    hypergraph = {"nodes": nodes, "hyperedges": hyperedges}
    return hypergraph

def solve_linear_program():
    model = Model("Linear_Programming_Example")
    
    query = examples.get_example(6)
    attributes = query["attributes"]
    relations = query["relations"]
    fds = query["fds"]
    
    hypergraph = create_hypergraph(attributes, relations)
    nodes = hypergraph["nodes"]
    hyperedges = hypergraph["hyperedges"]
    
    variables = {}
    for subset_size in range(1, len(attributes) + 1):
        for subset in itertools.combinations(attributes, subset_size):
            subset = list(subset)
            x = model.addVar(lb=0, name="".join(subset))
            variables[frozenset(subset)] = x
    
    #for e in hyperedges:
    #    name = "".join(e)
    #    x = model.addVar(lb=0, name=name)
    #    variables[e] = x
    
    objective = -sum([variables[frozenset(e)] for e in attributes]) + sum(variables[e] for e in hyperedges) #variables[frozenset(attributes)]
    model.setObjective(objective, GRB.MINIMIZE)
    
    for n in hyperedges:
        constraint = sum(variables[frozenset(e)] for e in n) <= 1
        model.addConstr(constraint)
    
    for n in nodes:
        constraint = sum(variables[frozenset(e)] for e in hyperedges if n in e) >= 1
        model.addConstr(constraint)
    
    for fd in fds:
        Y, x = fd[0], fd[1]
        Yx = Y + x
        model.addConstr(variables[frozenset(Yx)] == variables[frozenset(Y)], "fd_" + "".join(Y) + "->" + "".join(x))
    
    for x in attributes:
        Y = [y for y in attributes if y != x]
        model.addConstr(variables[frozenset(attributes)] - variables[frozenset(Y)] >= 0, name="Monotonicity")
        
        for j, l in itertools.combinations(attributes, 2):
            S = [x for x in attributes if x != j and x != l]
            for i in range(1, len(S) + 1):
                for subset in itertools.combinations(S, i):
                    subset = list(subset)
                    jS = subset + [j]
                    lS = subset + [l]
                    jlS = subset + [j, l]
                    model.addConstr(variables[frozenset(jS)] + variables[frozenset(lS)] - variables[frozenset(subset)] - variables[frozenset(jlS)] >= 0) #, name="Submodularity")
                    
            # Submodularity for those cases that subset S is empty (this reduces to subadditivity)
            for j, l in itertools.combinations(attributes, 2):
                model.addConstr(variables[frozenset([j])] + variables[frozenset([l])] - variables[frozenset([j, l])] >= 0) #, name="Subadditivity")

    model.write("Linear_Programming_Example.lp")
    model.optimize()

    if model.status == GRB.OPTIMAL:
        print("Optimal solution found!")
        print(f"Objective value = {model.objVal}")
        for v in model.getVars():
            print(f"{v.varName} = {v.x}")

solve_linear_program()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-11-26
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 229 rows, 15 columns and 744 nonzeros
Model fingerprint: 0xadf82edd
Academic license - for non-commercial use only - expires 2025-11-26
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 229 rows, 15 columns and 744 nonzeros
Model fingerprint: 0xadf82edd
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve re